Realizaremos un proceso de carga manual de datos en Databricks, con formatos .csv (8523 registros) y .json.(853 registros). Por fines demostrativos y mejorar la visualización se limitara a 15 registros.

**DATA 1:**\
Los datos pertenecen un registro de inventario de un supermercado. Al cual de con fines demostrativos, generaremos los siguientes requerimientos:

- Ajustar el nombre de una columna para claridad.
- Seleccionar tres columnas clave del inventario.
- Renombrar "Código del Producto" a "ID del Producto".
 - Filtrar productos por contenido graso Regular tipo "Soft Drinks" y peso, menor a 10 ml en "Tier 1 o Tier 2" con ubicación nula.
- Crear una columna para diferenciar este inventario de otros.
- Calcular el peso total vendido por producto.
- Unificar los tamaños del outlet en abreviaturas ("S", "M", "L").
- Cambiar el año de establecimiento de número a texto.
- Ordenar productos por peso y visibilidad, de mayor a menor.

**DATA 2:**\
Los datos pertenecen un registro de drivers en formato .JSON. Al cual de con fines demostraticvos, generaremos los siguientes requerimientos:
- Crear una columna con el nombre completo de cada jugador.
- Calcular su edad basandose en su fecha de nacimiento.

Data 1 Import 
Ingresaremos la data de manera manera manual en DBFS.

'/FileStore/tables/BigMart_Sales.csv'
'/FileStore/tables/drivers.json'

In [0]:
df = spark.read.format('csv').option('inferSchema', True).option('header', True).load('/FileStore/tables/BigMart_Sales.csv')

In [0]:
df.limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535


### Transformation

Read Schema

In [0]:
df.printSchema()

root
 |-- Item_Identifier: string (nullable = true)
 |-- Item_Weight: double (nullable = true)
 |-- Item_Fat_Content: string (nullable = true)
 |-- Item_Visibility: double (nullable = true)
 |-- Item_Type: string (nullable = true)
 |-- Item_MRP: double (nullable = true)
 |-- Outlet_Identifier: string (nullable = true)
 |-- Outlet_Establishment_Year: integer (nullable = true)
 |-- Outlet_Size: string (nullable = true)
 |-- Outlet_Location_Type: string (nullable = true)
 |-- Outlet_Type: string (nullable = true)
 |-- Item_Outlet_Sales: double (nullable = true)



DDL Schema\
Evaluamos y modificamos la columna Item_Weight a 'String'

In [0]:
new_ddl_schema = '''
                Item_Identifier: string,
                Item_Weight: string,
                Item_Fat_Content: string,
                Item_Visibility: double,
                Item_Type: string,
                Item_MRP: double,
                Outlet_Identifier: string,
                Outlet_Establishment_Year: integer,
                Outlet_Size: string,
                Outlet_Location_Type: string,
                Outlet_Type: string,
                Item_Outlet_Sales: double,
 '''

In [0]:
df_new_schema = spark.read.format('csv').option('new_ddl_schema', True).option('header', True).load('/FileStore/tables/BigMart_Sales.csv')

In [0]:
df_new_schema.limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
FDP36,10.395,Regular,0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535


Select\
Seleccionaremos sólo las columnas:
- Item_Identifier.
- Item_Type.
- Item_Weight.

In [0]:
from pyspark.sql.types import * 
from pyspark.sql.functions import *  

In [0]:
df.select(col('Item_Identifier'), col('Item_Weight'), col('Item_Type')).limit(15).display()
                   

Item_Identifier,Item_Weight,Item_Type
FDA15,9.3,Dairy
DRC01,5.92,Soft Drinks
FDN15,17.5,Meat
FDX07,19.2,Fruits and Vegetables
NCD19,8.93,Household
FDP36,10.395,Baking Goods
FDO10,13.65,Snack Foods
FDP10,null,Snack Foods
FDH17,16.2,Frozen Foods
FDU28,19.2,Frozen Foods


Entregar un alias a una columna

In [0]:
df.select(col('Item_Identifier').alias('Item_ID')).limit(15).display()

Item_ID
FDA15
DRC01
FDN15
FDX07
NCD19
FDP36
FDO10
FDP10
FDH17
FDU28


Filtrar data segun los siguientes requerimientos:
- Item_Fat_Content = 'Regular'
- Item_type = 'Soft Drinks' y peso menor a 10ml
- Ubicacion = 'Tier 1' o 'Tier 2' y el 'Outlet_Location_Size' sea nulo.

Requerimiento 1:

In [0]:
df.filter(col('Item_Fat_Content')== 'Regular').limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535
FDA03,18.5,Regular,0.045463773,Dairy,144.1102,OUT046,1997,Small,Tier 1,Supermarket Type1,2187.153
FDX32,15.1,Regular,0.1000135,Fruits and Vegetables,145.4786,OUT049,1999,Medium,Tier 1,Supermarket Type1,1589.2646
FDS46,17.6,Regular,0.047257328,Snack Foods,119.6782,OUT046,1997,Small,Tier 1,Supermarket Type1,2145.2076
FDP49,9.0,Regular,0.069088961,Breakfast,56.3614,OUT046,1997,Small,Tier 1,Supermarket Type1,1547.3192


Requerimiento 2:

In [0]:
df.filter((col('Item_Type')== 'Soft Drinks') & (col('Item_Weight')< 10)).limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
DRZ11,8.85,Regular,0.113123893,Soft Drinks,122.5388,OUT018,2009,Medium,Tier 3,Supermarket Type2,1609.9044
DRF49,7.27,Low Fat,0.071077939,Soft Drinks,114.2518,OUT046,1997,Small,Tier 1,Supermarket Type1,2618.5914
DRK12,9.5,LF,0.041878397,Soft Drinks,32.99,OUT035,2004,Small,Tier 2,Supermarket Type1,133.16
DRK01,7.63,Low Fat,0.06105276,Soft Drinks,95.4436,OUT035,2004,Small,Tier 2,Supermarket Type1,1418.154
DRF49,7.27,Low Fat,0.071222087,Soft Drinks,113.5518,OUT045,2002,null,Tier 2,Supermarket Type1,569.259
DRE60,9.395,Low Fat,0.159657596,Soft Drinks,224.972,OUT045,2002,null,Tier 2,Supermarket Type1,7696.648
DRI01,7.97,Low Fat,0.034452949,Soft Drinks,174.0422,OUT046,1997,Small,Tier 1,Supermarket Type1,2586.633
DRD37,9.8,Low Fat,0.013898123,Soft Drinks,46.506,OUT018,2009,Medium,Tier 3,Supermarket Type2,372.848
DRH13,8.575,Low Fat,0.023983258,Soft Drinks,106.328,OUT018,2009,Medium,Tier 3,Supermarket Type2,958.752


In [0]:
df.filter((col('Outlet_Size').isNull()) & (col('Outlet_Location_Type').isin('Tier 1', 'Tier 2'))).limit (15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535
NCD06,13.0,Low Fat,0.099887103,Household,45.906,OUT017,2007,null,Tier 2,Supermarket Type1,838.908
FDO23,17.85,Low Fat,0.0,Breads,93.1436,OUT045,2002,null,Tier 2,Supermarket Type1,2174.5028
NCP05,19.6,Low Fat,0.0,Health and Hygiene,153.3024,OUT045,2002,null,Tier 2,Supermarket Type1,2428.8384
FDV49,10.0,Low Fat,0.025879577,Canned,265.2226,OUT045,2002,null,Tier 2,Supermarket Type1,5815.0972
FDA43,10.895,Low Fat,0.065041581,Fruits and Vegetables,196.3794,OUT017,2007,null,Tier 2,Supermarket Type1,3121.2704
NCP18,12.15,Low Fat,0.028760013,Household,151.4708,OUT017,2007,null,Tier 2,Supermarket Type1,4815.0656
NCX54,9.195,Low Fat,0.048157338,Household,106.1622,OUT045,2002,null,Tier 2,Supermarket Type1,2117.244
FDV27,7.97,Regular,0.040071131,Meat,87.3514,OUT045,2002,null,Tier 2,Supermarket Type1,1062.6168


In [0]:
# Dado que solo encuentra en Tier 2, verificamos que no existan registro en Tier 1.

df.filter((col('Outlet_Size').isNull()) & (col('Outlet_Location_Type').isin('Tier 1'))).limit (15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales


Renombrar una columna


In [0]:
df = df.withColumnRenamed('Item_Identifier', 'Item_Id')


Agregar una columna personalizada (constante)

In [0]:
df = df.withColumn('Flag', lit('New'))

df.limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flag
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,New
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,New
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27,New
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38,New
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,New
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,New
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,New
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,New
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986,New
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535,New


Agregar una columna calculada (calculo matematico)\
'Item_Weight'*'Item_Outlet_Sales'


In [0]:
df = df.withColumn('multiply',col('Item_Weight')* col('Item_Outlet_Sales'))

df.limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flag,multiply
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,New,34736.7834
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,New,2625.062976
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27,New,36702.225
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38,New,14061.696
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,New,8882.717435999999
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,New,5785.948476
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528,New,4689.49572
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636,New,null
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986,New,17440.89732
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535,New,90442.272


Sustituir valores en un columna
'Outlet_Size'

In [0]:
df.withColumn('Outlet_Size', regexp_replace('Outlet_Size', 'Small', 'S')) \
    .withColumn('Outlet_Size', regexp_replace('Outlet_Size', 'Medium', 'M')) \
    .withColumn('Outlet_Size', regexp_replace('Outlet_Size', 'High', 'L')).limit (15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flag,multiply
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,M,Tier 1,Supermarket Type1,3735.138,New,34736.7834
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,M,Tier 3,Supermarket Type2,443.4228,New,2625.062976
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,M,Tier 1,Supermarket Type1,2097.27,New,36702.225
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38,New,14061.696
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,L,Tier 3,Supermarket Type1,994.7052,New,8882.717435999999
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,M,Tier 3,Supermarket Type2,556.6088,New,5785.948476
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,L,Tier 3,Supermarket Type1,343.5528,New,4689.49572
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,M,Tier 3,Supermarket Type3,4022.7636,New,null
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986,New,17440.89732
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535,New,90442.272


Modificar tipo de columna 'Outlet_Establishment_Year' de numerico a texto

In [0]:
df = df.withColumn('Outlet_Establishment_Year', col('Outlet_Establishment_Year').cast(StringType()))

In [0]:
df.limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38
NCD19,8.93,Low Fat,0.0,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
FDO10,13.65,Regular,0.012741089,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
FDP10,null,Low Fat,0.127469857,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535


Ordernar productos por peso y visibilidad [DESC](url) (Sort/GroupBy)

In [0]:
df.sort(['Item_Weight','Item_Visibility'],ascending = [0,0]).limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
FDR07,21.35,Low Fat,0.130127365,Fruits and Vegetables,96.2094,OUT010,1998,null,Tier 3,Grocery Store,190.4188
FDC02,21.35,Low Fat,0.115194717,Canned,258.3278,OUT010,1998,null,Tier 3,Grocery Store,520.6556
FDR07,21.35,Low Fat,0.078060605,Fruits and Vegetables,96.0094,OUT018,2009,Medium,Tier 3,Supermarket Type2,380.8376
FDC02,21.35,Low Fat,0.069102831,Canned,259.9278,OUT018,2009,Medium,Tier 3,Supermarket Type2,6768.5228
FDC02,21.35,Low Fat,0.068822477,Canned,258.3278,OUT046,1997,Small,Tier 1,Supermarket Type1,7028.8506
FDC02,21.35,Low Fat,0.068809463,Canned,258.5278,OUT035,2004,Small,Tier 2,Supermarket Type1,5206.556
FDC02,21.35,Low Fat,0.068765205,Canned,260.4278,OUT013,1987,High,Tier 3,Supermarket Type1,3644.5892
FDA45,21.25,Low Fat,0.156012631,Snack Foods,177.337,OUT018,2009,Medium,Tier 3,Supermarket Type2,2822.992
FDA45,21.25,Low Fat,0.155694794,Snack Foods,177.637,OUT045,2002,null,Tier 2,Supermarket Type1,2999.429
FDA45,21.25,Low Fat,0.155350299,Snack Foods,178.237,OUT035,2004,Small,Tier 2,Supermarket Type1,529.311


Eliminar duplicados

In [0]:
df.dropDuplicates().limit(15).display()

Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,Flag,multiply
FDA15,9.3,Low Fat,0.016047301,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.138,New,34736.7834
FDN15,17.5,Low Fat,0.016760075,Meat,141.618,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.27,New,36702.225
DRC01,5.92,Regular,0.019278216,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,New,2625.062976
FDH17,16.2,Regular,0.016687114,Frozen Foods,96.9726,OUT045,2002,null,Tier 2,Supermarket Type1,1076.5986,New,17440.89732
FDS46,17.6,Regular,0.047257328,Snack Foods,119.6782,OUT046,1997,Small,Tier 1,Supermarket Type1,2145.2076,New,37755.65376000001
FDA03,18.5,Regular,0.045463773,Dairy,144.1102,OUT046,1997,Small,Tier 1,Supermarket Type1,2187.153,New,40462.3305
FDX32,15.1,Regular,0.1000135,Fruits and Vegetables,145.4786,OUT049,1999,Medium,Tier 1,Supermarket Type1,1589.2646,New,23997.89546
FDP36,10.395,Regular,0.0,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088,New,5785.948476
FDU28,19.2,Regular,0.09444959,Frozen Foods,187.8214,OUT017,2007,null,Tier 2,Supermarket Type1,4710.535,New,90442.272
FDX07,19.2,Regular,0.0,Fruits and Vegetables,182.095,OUT010,1998,null,Tier 3,Grocery Store,732.38,New,14061.696


**Data 2**

JSON Reading

In [0]:
df_json = spark.read.format('json').option('inferSchema', True).option('multiLine', False).option('header', True).load('/FileStore/tables/drivers.json')

In [0]:
df_json.limit(15).display()

code,dob,driverId,driverRef,name,nationality,number,url
HAM,1985-01-07,1,hamilton,"List(Lewis, Hamilton)",British,44,http://en.wikipedia.org/wiki/Lewis_Hamilton
HEI,1977-05-10,2,heidfeld,"List(Nick, Heidfeld)",German,\N,http://en.wikipedia.org/wiki/Nick_Heidfeld
ROS,1985-06-27,3,rosberg,"List(Nico, Rosberg)",German,6,http://en.wikipedia.org/wiki/Nico_Rosberg
ALO,1981-07-29,4,alonso,"List(Fernando, Alonso)",Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso
KOV,1981-10-19,5,kovalainen,"List(Heikki, Kovalainen)",Finnish,\N,http://en.wikipedia.org/wiki/Heikki_Kovalainen
NAK,1985-01-11,6,nakajima,"List(Kazuki, Nakajima)",Japanese,\N,http://en.wikipedia.org/wiki/Kazuki_Nakajima
BOU,1979-02-28,7,bourdais,"List(Sébastien, Bourdais)",French,\N,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais
RAI,1979-10-17,8,raikkonen,"List(Kimi, Räikkönen)",Finnish,7,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen
KUB,1984-12-07,9,kubica,"List(Robert, Kubica)",Polish,88,http://en.wikipedia.org/wiki/Robert_Kubica
GLO,1982-03-18,10,glock,"List(Timo, Glock)",German,\N,http://en.wikipedia.org/wiki/Timo_Glock


Nueva columna con nombre completo

In [0]:
df_json = df_json.withColumn("full_name", concat_ws(" ", "name.forename", "name.surname"))

df_json.limit(15).display()

code,dob,driverId,driverRef,name,nationality,number,url,full_name
HAM,1985-01-07,1,hamilton,"List(Lewis, Hamilton)",British,44,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis Hamilton
HEI,1977-05-10,2,heidfeld,"List(Nick, Heidfeld)",German,\N,http://en.wikipedia.org/wiki/Nick_Heidfeld,Nick Heidfeld
ROS,1985-06-27,3,rosberg,"List(Nico, Rosberg)",German,6,http://en.wikipedia.org/wiki/Nico_Rosberg,Nico Rosberg
ALO,1981-07-29,4,alonso,"List(Fernando, Alonso)",Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando Alonso
KOV,1981-10-19,5,kovalainen,"List(Heikki, Kovalainen)",Finnish,\N,http://en.wikipedia.org/wiki/Heikki_Kovalainen,Heikki Kovalainen
NAK,1985-01-11,6,nakajima,"List(Kazuki, Nakajima)",Japanese,\N,http://en.wikipedia.org/wiki/Kazuki_Nakajima,Kazuki Nakajima
BOU,1979-02-28,7,bourdais,"List(Sébastien, Bourdais)",French,\N,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais,Sébastien Bourdais
RAI,1979-10-17,8,raikkonen,"List(Kimi, Räikkönen)",Finnish,7,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,Kimi Räikkönen
KUB,1984-12-07,9,kubica,"List(Robert, Kubica)",Polish,88,http://en.wikipedia.org/wiki/Robert_Kubica,Robert Kubica
GLO,1982-03-18,10,glock,"List(Timo, Glock)",German,\N,http://en.wikipedia.org/wiki/Timo_Glock,Timo Glock


In [0]:
df_json = df_json.withColumn("age",(datediff(current_date(), col("dob")) / 365).cast("int"))

df_json.limit(15).display()

code,dob,driverId,driverRef,name,nationality,number,url,full_name,age
HAM,1985-01-07,1,hamilton,"List(Lewis, Hamilton)",British,44,http://en.wikipedia.org/wiki/Lewis_Hamilton,Lewis Hamilton,40
HEI,1977-05-10,2,heidfeld,"List(Nick, Heidfeld)",German,\N,http://en.wikipedia.org/wiki/Nick_Heidfeld,Nick Heidfeld,47
ROS,1985-06-27,3,rosberg,"List(Nico, Rosberg)",German,6,http://en.wikipedia.org/wiki/Nico_Rosberg,Nico Rosberg,39
ALO,1981-07-29,4,alonso,"List(Fernando, Alonso)",Spanish,14,http://en.wikipedia.org/wiki/Fernando_Alonso,Fernando Alonso,43
KOV,1981-10-19,5,kovalainen,"List(Heikki, Kovalainen)",Finnish,\N,http://en.wikipedia.org/wiki/Heikki_Kovalainen,Heikki Kovalainen,43
NAK,1985-01-11,6,nakajima,"List(Kazuki, Nakajima)",Japanese,\N,http://en.wikipedia.org/wiki/Kazuki_Nakajima,Kazuki Nakajima,39
BOU,1979-02-28,7,bourdais,"List(Sébastien, Bourdais)",French,\N,http://en.wikipedia.org/wiki/S%C3%A9bastien_Bourdais,Sébastien Bourdais,45
RAI,1979-10-17,8,raikkonen,"List(Kimi, Räikkönen)",Finnish,7,http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen,Kimi Räikkönen,45
KUB,1984-12-07,9,kubica,"List(Robert, Kubica)",Polish,88,http://en.wikipedia.org/wiki/Robert_Kubica,Robert Kubica,40
GLO,1982-03-18,10,glock,"List(Timo, Glock)",German,\N,http://en.wikipedia.org/wiki/Timo_Glock,Timo Glock,42
